# Scraping OpenTable

In [97]:
# import libraries
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta 

In [10]:
# accessing the website and pulling data
url = 'https://www.opentable.com/r/american-son-washington'
response = requests.get(url)
print(response.status_code)
page_tree = BeautifulSoup(response.content, 'html.parser')


200


In [14]:
page_tree.main.div.p

<p class="_3fd7bece"><span>Top Tags</span>: </p>

In [3]:
divs = page_tree.main.div.find_all('div')

In [11]:
len(divs)

1049

In [40]:
# getting all reviews
all_reviews = []
for review in page_tree.select('p[id*="review-body-"]'):
    all_reviews.append(review.get_text())
    #print(review.get_text())
    #print()
print(len(all_reviews))
#all_reviews

40


In [115]:
# getting dates
# referenced this site: https://www.dataquest.io/blog/python-datetime-tutorial/https://www.dataquest.io/blog/python-datetime-tutorial/
all_dates_span = page_tree.find_all("span", {"class": "oc-reviews-47b8de40"})
all_dates = []
for date in all_dates_span:
    curr = date.get_text()
    if 'on' in curr:
        curr = curr[9:]
        curr = datetime.strptime(curr, "%B %d, %Y")
        # all_dates.append('' + str(curr.year) + '-' + str(curr.month) + '-' + str(curr.day))
        #print(my_date.year)
    else:
        curr = curr[6:8]
        curr = int(curr.strip())
        now = datetime.now()
        # curr_str = '' + str(curr) + str(now.month) + str(now.year) + ''
        # curr_date = datetime.strptime(curr_str, "%d%m%Y")
        curr = now - timedelta(days=curr+1)
    all_dates.append(curr.date())
all_dates

[datetime.date(2021, 3, 11),
 datetime.date(2021, 2, 27),
 datetime.date(2021, 2, 22),
 datetime.date(2021, 2, 21),
 datetime.date(2021, 2, 20),
 datetime.date(2021, 2, 7),
 datetime.date(2021, 1, 30),
 datetime.date(2021, 1, 29),
 datetime.date(2020, 11, 26),
 datetime.date(2020, 11, 26),
 datetime.date(2020, 11, 16),
 datetime.date(2020, 10, 23),
 datetime.date(2020, 10, 11),
 datetime.date(2020, 10, 3),
 datetime.date(2020, 9, 26),
 datetime.date(2020, 9, 5),
 datetime.date(2020, 9, 5),
 datetime.date(2020, 8, 29),
 datetime.date(2020, 8, 29),
 datetime.date(2020, 8, 23),
 datetime.date(2020, 8, 23),
 datetime.date(2020, 8, 23),
 datetime.date(2020, 8, 22),
 datetime.date(2020, 3, 4),
 datetime.date(2020, 3, 3),
 datetime.date(2020, 2, 29),
 datetime.date(2020, 2, 28),
 datetime.date(2020, 2, 27),
 datetime.date(2020, 2, 25),
 datetime.date(2020, 2, 25),
 datetime.date(2020, 2, 22),
 datetime.date(2020, 2, 21),
 datetime.date(2020, 2, 17),
 datetime.date(2020, 2, 16),
 datetime.date

In [116]:
review_df = pd.DataFrame(all_reviews, columns = ['review']) 
review_df['date'] = all_dates
review_df

,review,date
0,The calamari dish was Luke warm despite the co...,2021-03-11
1,"Great staff, very accommodating. At one point ...",2021-02-27
2,"Ambiance was great, trendy yet retro. Cute spo...",2021-02-22
3,"Food was average. The menu read a lot more ""in...",2021-02-21
4,Food was not good. Not much options to pick from,2021-02-20
5,Blah is the best word to describe the brunch d...,2021-02-07
6,The service was overall disappointing. Our ser...,2021-01-30
7,We waited 55 minutes for the first course to c...,2021-01-29
8,I would definitely come back again!!! Ms. Tiff...,2020-11-26
9,Such great service & good covid precautions.,2020-11-26


In [118]:
opentable_reviewss = review_df.to_csv('/Ascend/opentable_revi')